In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from firebase import FirebaseManager 
from mongo import MongoDBManager
from selenium.webdriver.common.keys import Keys
import os
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from datetime import datetime

# web Scraping Arnauld [DéfIA]

## Site 1 : Reuter

In [3]:
load_dotenv()
uri = os.getenv('MONGODB_URI')
database_name = os.getenv('DATABASE_NAME')
mongo_manager = MongoDBManager(uri, database_name)
stats = mongo_manager.get_database_storage_infos()

Taille des données : 215.68148612976074 Mo
Taille des données en pourcentage: 42.125290259718895 %


In [4]:
# Liste des indicateurs qui nous intéressent

indicator_list = [
    "inflation", 
    "unemployment rate", 
    "money supply", 
    "import-export", 
    "GDP", 
    "production volume", 
    "3-month treasury rate", 
    "exchange rate US/CAD", 
    "balance of payment", 
    "Real GDP"
]

In [5]:
import random
from selenium.webdriver.chrome.options import Options


USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.81",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14.4; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux i686; rv:124.0) Gecko/20100101 Firefox/124.0",
]

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument(f'user-agent={random.choice(USER_AGENTS)}')


def random_sleep(min_time=1, max_time=10):
    time.sleep(random.uniform(min_time, max_time))


In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-extensions")


driver = webdriver.Chrome(chrome_options)
for indicator in tqdm(indicator_list):
    driver.get(f"https://www.reuters.com/site-search/?query={indicator}")
    links_of_articles = []
    current_list_of_li=driver.find_elements(By.XPATH,'//div[@class="search-results__sectionContainer__34n_c"]/ul/li')

    for li in current_list_of_li:
        link = li.find_element(By.XPATH, './div').get_attribute('href')
        time = li.find_element(By.XPATH, './div/div/time').get_attribute('datetime')
        date_obj = datetime.strptime(time, '%Y-%m-%dT%H:%M:%SZ')
        links_of_articles.append({'link': link, 'time': date_obj})    

    filtered_links_of_articles = []

    if len(links_of_articles) >10 :
        for link in links_of_articles:
            filtered_list = []
            if date_obj.year == 2024 and date_obj.month >= 8 :       
                filtered_list.append({'link': link, 'time': date_obj})
            else:
                print(f"Article trop vieux ({date_obj})")
            
            if len(filtered_list) == 0:
                filtered_list.extend(links_of_articles[:10])
            if len(filtered_list) < 10:
                not_in_filtered_list = [link for link in links_of_articles if link not in filtered_list]
                filtered_list.extend(not_in_filtered_list[:min(len(not_in_filtered_list), 10 - len(filtered_list))])
            filtered_links_of_articles.extend(filtered_list)
    else:
        filtered_links_of_articles.extend(links_of_articles)    

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
for link in filtered_links_of_articles:
    driver.get(link['link'])
    random_sleep()
    paragraphs = driver.find_elements(By.XPATH, '//div[contains(@data-testid, "paragraph")]')
    text=""
    for paragraph in paragraphs:
        text += driver.execute_script("return arguments[0].innerText;", paragraph)+"."
    mongo_manager.add_data_to_mongodb("Articles", link['time'], text)
    random_sleep()